In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

data = pd.read_csv('data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day_of_week'] = data['Date'].dt.dayofweek
data['Day'] = data['Date'].dt.day
data['Public Holiday'] = data['Public Holiday'].fillna(0)
data['Public Holiday'] = data['Public Holiday'].apply(lambda x: 1 if x else 0)
data['Event'] = data['Event'].apply(lambda x: 1 if x == 'TRUE' else 0)

X = data[['Day_of_week', 'Day', 'Month', 'Year','Event', 'Public Holiday',
          'India_Reservation']]
y = data[['Customers_Chinese', 'Customers_India']]

busy_threshold = 100
def is_busy(customers_prediction):
    return customers_prediction > busy_threshold

X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 7)

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train,y_train)

predictions = pd.DataFrame(model.predict(X_test), columns=['Predicted_Customers_Chinese', 'Predicted_Customers_India'])
X_test['Date'] = X_test.apply(lambda row: f"{row['Day']}/{row['Month']}/{row['Year']}", axis=1)
predictions['Date']= pd.to_datetime(X_test['Date'].values, format='%d/%m/%Y')
predictions['Day_of_week'] = predictions['Date'].dt.day_name()
X_test.reset_index(drop=True, inplace=True)
predictions['Public Holiday'] = X_test['Public Holiday']
predictions['Chinese_Buffet_Busy'] = predictions['Predicted_Customers_Chinese'].apply(is_busy)
predictions['Indian_Buffet_Busy'] = predictions['Predicted_Customers_India'].apply(is_busy)
predictions = predictions[['Date', 'Day_of_week', 'Public Holiday', 'Predicted_Customers_Chinese', 'Chinese_Buffet_Busy', 'Predicted_Customers_India', 'Indian_Buffet_Busy']]

print(predictions)


        Date Day_of_week  Public Holiday  Predicted_Customers_Chinese  \
0 2021-02-25    Thursday               0                         87.0   
1 2023-03-27      Monday               0                        105.0   
2 2023-12-30    Saturday               0                        148.0   
3 2022-04-15      Friday               1                        123.0   
4 2022-01-08    Saturday               0                        101.0   
5 2023-10-28    Saturday               0                        127.0   
6 2022-08-06    Saturday               0                        176.0   

   Chinese_Buffet_Busy  Predicted_Customers_India  Indian_Buffet_Busy  
0                False                       83.0               False  
1                 True                       86.0               False  
2                 True                        0.0               False  
3                 True                      199.0                True  
4                 True                       83.0      

<ipython-input-67-7b081987651b>:6: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Date'] = pd.to_datetime(data['Date'])


In [71]:
predictions.to_csv('2024 predictions.csv')